<a href="https://colab.research.google.com/github/AsifAlFaisal/ecmwf-climate-data-proecss-gee/blob/main/bangladesh_climate_data_analysis_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading necessary libraries

In [ ]:
import ee
import geemap

## Connecting Colab with Google Earth Engine

In [ ]:
# Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6RRt1oSWpEwR7bs9lN4W-FUtlTKbJdpdctuHjKNfHyo&tc=08yAPMchF1MDP31JrZF_k24ijoOAD9UqGsN7vS4yqDo&cc=Y8kizf3EnaNrTPQXHbBYohK1Ti3TuTEGSnmjfhuKSi4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkATt-3kJ1guo07qhjfyIQa_k9Myji9hHrFoYpawJzU2ctd_IDxGno

Successfully saved authorization token.


## Defining your Region of Interest (ROI)

- Visit to https://geojson.io/
- Select the area you are interested in
- Get the bounding box coordinates of the selected area

In [ ]:
# got from https://geojson.io/
ROI_BOUNDS = [
          [
            [
              90.40844385465113,
              24.035066559554096
            ],
            [
              90.40844385465113,
              24.00887582501791
            ],
            [
              90.45827768232465,
              24.00887582501791
            ],
            [
              90.45827768232465,
              24.035066559554096
            ],
            [
              90.40844385465113,
              24.035066559554096
            ]
          ]
        ]

## Defining filtering parameters

In [ ]:
# Dataset Filter Parameters
ROI = ee.Geometry.Polygon(ROI_BOUNDS)
START_DATE = '2023-01-01'
END_DATE = '2023-05-31'

## Connect ECMWF Data Store with Google Earth Engine using Colab
- Data storage link inside GEE:
- ERA5-Land Daily Aggregated - ECMWF Climate Reanalysis
- https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR

In [ ]:
temp_2m = (
    ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
    .filterBounds(ROI)
    .select('temperature_2m')
    .filter(ee.Filter.date(START_DATE, END_DATE))
)

In [ ]:
soil_temp_l1 = (
    ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
    .filterBounds(ROI)
    .select('soil_temperature_level_1')
    .filter(ee.Filter.date(START_DATE, END_DATE))
)

In [ ]:
total_precip = (
    ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
    .filterBounds(ROI)
    .select('total_precipitation_sum')
    .filter(ee.Filter.date(START_DATE, END_DATE))
)

## Setting up visualization parameters
- Define a function for visualization parameters
- Define a function to visualize the data

In [ ]:
def viz_param_function(min_range, max_range):
  '''
  e.g. range:
    temperature: 240 - 304
    precipitation: 0 - 0.1
  '''
  vis_params = {
      'min': min_range,
      'max': max_range,
      'palette': [
      '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff',
      '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00',
      'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff',],
      }
  return vis_params


# Data Visualization function
def data_visualize(dataset, roi, min_range, max_range, zoom, name=""):
  import geemap
  # Calculate the centroid of the ROI
  centroid = roi.centroid()

  # Get the coordinates of the centroid
  centroid_coords = centroid.coordinates().getInfo()[::-1]

  # Create a Map instance with the centroid as the center
  m = geemap.Map(center=centroid_coords, zoom=zoom)
  m.add_ee_layer(dataset,
                 viz_param_function(min_range, max_range),
                 "")

  return m

## Visualize temperature above 2m at the ROI

In [ ]:
data_visualize(temp_2m, ROI, 240, 304, 10)

## Visualize total precipitation at the ROI

In [ ]:
data_visualize(total_precip, ROI, 0, 0.1, 10)

## Visualize soil temperature at the ROI

In [ ]:
data_visualize(soil_temp_l1, ROI, 240, 304, 10)

## Data Loading and Extraction
- Load your gps coordinates within your selected ROI
- Extract required climate data values at those coordinates

In [ ]:
import pandas as pd

df = pd.read_csv("/content/sample_coords.csv")
df.head()

,Point,Longitude,Latitude
0,pt1,90.432174,24.014837
1,pt2,90.422089,24.018630
2,pt3,90.444830,24.020256
3,pt4,90.430988,24.023868
4,pt5,90.450368,24.030009


## Function to extract mean pixel values for a labeled location

In [ ]:
# Function to extract mean pixel values for a labeled location
def extract_mean_pixel_value(dataset, lat, lon, band_name):

    # Create a point of interest geometry for the labeled location
    poi = ee.Geometry.Point(lon, lat)

    # Calculate the mean pixel values for the bands of interest at the labeled location
    mean_values = dataset.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=poi,
        scale=10)

    # Extract the mean pixel values for the bands
    #values = [mean_values.get(band).getInfo() for band in bands]

    return mean_values.get(band_name).getInfo()

## Showing example by extracting pixel value at a single coordinate

In [ ]:
extract_mean_pixel_value(soil_temp_l1, lat=23.6879120711, lon=89.8537443651, band_name = 'soil_temperature_level_1')

300.8679838858711

## Get the pixel values for your file

In [ ]:
df['temperature_2m'] = df.apply(lambda row: extract_mean_pixel_value(temp_2m,
                                                                     lat = row["Latitude"],
                                                                     lon = row["Longitude"],
                                                                     band_name='temperature_2m'),
                                axis=1)

In [ ]:
df['soil_temperature_level_1'] = df.apply(lambda row: extract_mean_pixel_value(soil_temp_l1,
                                                                     lat = row["Latitude"],
                                                                     lon = row["Longitude"],
                                                                     band_name='soil_temperature_level_1'),
                                axis=1)

In [ ]:
df['total_precipitation_sum'] = df.apply(lambda row: extract_mean_pixel_value(total_precip,
                                                                     lat = row["Latitude"],
                                                                     lon = row["Longitude"],
                                                                     band_name='total_precipitation_sum'),
                                axis=1)

In [ ]:
df.head()

,Point,Longitude,Latitude,temperature_2m,soil_temperature_level_1,total_precipitation_sum
0,pt1,90.432174,24.014837,297.758162,299.759486,0.002886
1,pt2,90.422089,24.018630,297.758162,299.759486,0.002886
2,pt3,90.444830,24.020256,297.758162,299.759486,0.002886
3,pt4,90.430988,24.023868,297.758162,299.759486,0.002886
4,pt5,90.450368,24.030009,297.818196,299.895879,0.002855


## Extract the climate data to csv format

In [ ]:
df.to_csv("sample_coords_output.csv", index = False)